In [11]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [12]:
# Step 1: Create an imbalanced binary classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100]))

In [13]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

#### Handle class imbalance

In [14]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)
np.unique(y_train_res, return_counts=True)

(array([0, 1]), array([619, 619]))

### Track Experiments

In [15]:
models = [
    (
        "Logistic Regression", 
        {"C": 1, "solver": 'lbfgs'},
        LogisticRegression(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "Random Forest", 
        {"n_estimators": 30, "max_depth": 3},
        RandomForestClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier With SMOTE",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train_res, y_train_res),
        (X_test, y_test)
    )
]

In [16]:
reports = []

for model_name, params, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]
    
    model.set_params(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

In [17]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost

In [18]:
# Initialize MLflow

mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("deployment model")
for i, element in enumerate(models):
    model_name = element[0]
    params = element[1]
    model = element[2]
    report = reports[i]
    
    with mlflow.start_run(run_name=model_name):        
        mlflow.log_params(params)
        mlflow.log_metrics({
            'accuracy': report['accuracy'],
            'recall_class_1': report['1']['recall'],
            'recall_class_0': report['0']['recall'],
            'f1_score_macro': report['macro avg']['f1-score']
        })  
        
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")  

2025/11/04 19:37:44 INFO mlflow.tracking.fluent: Experiment with name 'deployment model' does not exist. Creating a new experiment.
2025/11/04 19:37:44 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/04 19:37:50 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/11/04 19:37:53 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run Logistic Regression at: http://localhost:5000/#/experiments/630371390846440576/runs/6b5130c970df4c679d1c9382a5eaa581
🧪 View experiment at: http://localhost:5000/#/experiments/630371390846440576


2025/11/04 19:37:57 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/11/04 19:37:58 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run Random Forest at: http://localhost:5000/#/experiments/630371390846440576/runs/b17e35751e694a54b6a6ff5c06dfc058
🧪 View experiment at: http://localhost:5000/#/experiments/630371390846440576


2025/11/04 19:38:05 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/11/04 19:38:07 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run XGBClassifier at: http://localhost:5000/#/experiments/630371390846440576/runs/0707a34bb3754e5197f1f7859733c0f0
🧪 View experiment at: http://localhost:5000/#/experiments/630371390846440576


2025/11/04 19:38:15 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier With SMOTE at: http://localhost:5000/#/experiments/630371390846440576/runs/956a37a44a954014b23692b315d91f47
🧪 View experiment at: http://localhost:5000/#/experiments/630371390846440576


### Register the Model

In [20]:
import mlflow

# Step 1 — Set your model name
model_name = "XGB-Smote"

# Step 2 — Paste your actual Run ID directly (not with input)
run_id = "956a37a44a954014b23692b315d91f47"  # replace with your actual run_id

# Step 3 — Build model URI correctly
model_uri = f"runs:/{run_id}/model"

# Step 4 — Register the model (no need to start a new run)
mlflow.register_model(
    model_uri=model_uri,
    name=model_name
)


Registered model 'XGB-Smote' already exists. Creating a new version of this model...
2025/11/04 19:39:21 WARNING mlflow.tracking._model_registry.fluent: Run with id 956a37a44a954014b23692b315d91f47 has no artifacts at artifact path 'model', registering model based on models:/m-185bb8d4194a4496bad340214da0ac2f instead
2025/11/04 19:39:21 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XGB-Smote, version 1
Created version '1' of model 'XGB-Smote'.


<ModelVersion: aliases=[], creation_timestamp=1762265361245, current_stage='None', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='', last_updated_timestamp=1762265361245, metrics=None, model_id=None, name='XGB-Smote', params=None, run_id='956a37a44a954014b23692b315d91f47', run_link='', source='models:/m-185bb8d4194a4496bad340214da0ac2f', status='READY', status_message=None, tags={}, user_id='', version='1'>

### Load the Model

In [21]:
model_name = 'XGB-Smote'
model_version = 1
model_uri = f"models:/{model_name}/{model_version}"

loaded_model = mlflow.xgboost.load_model(model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

array([0, 0, 0, 0])

### Transition the Model to Production server

In [27]:
import mlflow
from mlflow import MlflowClient

# Set your MLflow tracking server URI
mlflow.set_tracking_uri("http://127.0.0.1:5000")

client = MlflowClient()

# Source model info
source_model_name = "XGB-Smote"
source_model_version = 1   # replace with the version you want to copy

# Destination model name
production_model_name = "finalproduction"

# Build the source model URI correctly
src_model_uri = f"models:/{source_model_name}/{source_model_version}"

# Copy the model version to a new registered model
client.copy_model_version(
    src_model_uri=src_model_uri,
    dst_name=production_model_name
)

print(f"✅ Successfully copied {source_model_name} v{source_model_version} → {production_model_name}")


✅ Successfully copied XGB-Smote v1 → finalproduction


Successfully registered model 'finalproduction'.
Copied version '1' of model 'XGB-Smote' to version '1' of model 'finalproduction'.


In [ ]:
'''
# ignore below code

model_version = 2
prod_model_uri = f"models:/{production_model_name}@productionserver2"

loaded_model = mlflow.xgboost.load_model(prod_model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]
'''

'\n# ignore below code\n\nmodel_version = 2\nprod_model_uri = f"models:/{production_model_name}@productionserver2"\n\nloaded_model = mlflow.xgboost.load_model(prod_model_uri)\ny_pred = loaded_model.predict(X_test)\ny_pred[:4]\n'

In [ ]:
import mlflow
print("Tracking URI:", mlflow.get_tracking_uri())



Tracking URI: file:///c:/Users/nnaya/Desktop/JOB/project/MLOPS/MLOPS-main/mlruns
